# 3. Data Preparation

Tahap Data Preparation bertujuan untuk menyiapkan data agar siap digunakan pada proses modeling. Pada tahap ini dilakukan seleksi fitur, transformasi data time series, serta pembagian data train dan test berdasarkan waktu.

## 3.1 Load dan Validasi Data

In [ ]:
import pandas as pd

# Load dataset
data = pd.read_csv('/mnt/data/3ed03445-3971-4d35-8f52-4845f6854f82.csv')

# Konversi kolom tanggal
data['date'] = pd.to_datetime(data['date'])

# Urutkan data berdasarkan tanggal
data = data.sort_values('date')

# Set date sebagai index
data = data.set_index('date')

data.head()

Dataset telah berada dalam format time series yang benar dengan index berupa tanggal.

## 3.2 Seleksi Fitur

Pada proyek ini digunakan pendekatan **price-only forecasting**. Oleh karena itu, hanya variabel `close` yang digunakan sebagai fitur utama.

Pendekatan ini dipilih untuk mengevaluasi sejauh mana harga historis saja mampu memprediksi harga Bitcoin di masa depan.

In [ ]:
# Seleksi fitur
df = data[['close']].copy()

df.head()

## 3.3 Train-Test Split Berbasis Waktu

Pembagian data dilakukan secara kronologis untuk menghindari data leakage. Data sebelum tanggal split digunakan sebagai data latih, sedangkan data setelahnya digunakan sebagai data uji.

**Tanggal split:** 25 Juni 2018

Tanggal ini dipilih karena berada setelah periode koreksi besar harga Bitcoin.

In [ ]:
# Train-test split
split_date = '2018-06-25'

train = df.loc[df.index <= split_date]
test  = df.loc[df.index > split_date]

train.shape, test.shape

## 3.4 Scaling Data

Scaling dilakukan untuk menyesuaikan rentang data dengan kebutuhan model LSTM. Min-Max Scaling dipilih karena umum digunakan pada neural network dan menjaga bentuk distribusi data.

In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

# Scaling data train
train_scaled = scaler.fit_transform(train)

train_scaled[:5]

## 3.5 Transformasi ke Supervised Learning

Data time series diubah menjadi format supervised learning dengan pendekatan satu langkah ke depan (t → t+1).

Pendekatan ini bertujuan untuk memprediksi harga penutupan hari berikutnya berdasarkan harga hari ini.

In [ ]:
# Membentuk X dan y
X_train = train_scaled[:-1]
y_train = train_scaled[1:]

# Reshape untuk LSTM: [samples, timesteps, features]
X_train = X_train.reshape((X_train.shape[0], 1, 1))

X_train.shape, y_train.shape

## 3.6 Output Tahap Data Preparation

Hasil dari tahap Data Preparation adalah:
- Data latih dan data uji yang terpisah secara kronologis
- Data yang telah di-scale
- Dataset dalam format supervised learning yang siap digunakan untuk modeling LSTM

Tahap selanjutnya adalah **Modeling**, di mana model LSTM akan dibangun dan dilatih menggunakan data yang telah dipersiapkan.